In [ ]:
import torch
import numpy as np
import pandas as pd
import torchvision.transforms as transforms
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Exploring Dataset

In [ ]:
# load dataset
train_data = pd.read_csv('../input/digit-recognizer/train.csv')
test_data = pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
print("Total number of samples",len(train_data),"\nTotal pixels: ", len(train_data.columns))

In [ ]:
train_data.head()

In [ ]:
class Get_Data(torch.utils.data.Dataset):
    def __init__(self, data, transform=transforms.Compose([transforms.ToPILImage(),
                                                                      transforms.RandomRotation(0, 0.5),
                                                                      transforms.RandomHorizontalFlip(p=0.5),
                                                                      transforms.ToTensor(),
                                                                      transforms.Normalize(mean=(0.5,), std=(0.5,))])):
        
        self.data = data
        if len(data.columns) == len(train_data.columns):
            self.X = data.iloc[:,1:].values.reshape((-1,28,28)).astype(np.float32)[:,:,:,None]
            self.X = np.multiply(self.X, 1.0/255.0)
            self.y = torch.from_numpy(data.iloc[:,0].values)
        else : 
            self.X = data.values.reshape((-1,28,28)).astype(np.float32)[:,:,:,None]
            self.X = np.multiply(self.X, 1.0/255.0)
            self.y = None
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        if self.y is not None:
            return self.transform(self.X[index]), self.y[index]
        else:
            return self.transform(self.X[index])


In [ ]:
train_df = Get_Data(train_data)
test_df = Get_Data(test_data)


In [ ]:
num_workers = 0
valid_size = 0.1
batch_size = 128
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_df, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_df, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_df, batch_size=batch_size, 
    num_workers=num_workers)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
    
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap=matplotlib.cm.binary)
    # print out the correct label for each image
    # .item() gets the value contained in a Tensor
    ax.set_title(str(labels[idx].item()))

**Viewing Image in More Detail**

In [ ]:
img = np.squeeze(images[1])

fig = plt.figure(figsize = (12,12)) 
ax = fig.add_subplot(111)
ax.imshow(img, cmap='gray')
width, height = img.shape
thresh = img.max()/2.5
for x in range(width):
    for y in range(height):
        val = round(img[x][y],2) if img[x][y] >=0 else 0
        ax.annotate(str(val), xy=(y,x),
                    horizontalalignment='center',
                    verticalalignment='center',
                    color='white' if img[x][y]<thresh else 'black')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.features =nn.Sequential(
            nn.Conv2d(1,64,kernel_size = 3, stride = 1 , padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64,128,kernel_size = 3, stride = 1 , padding = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.MaxPool2d(2,2),
            nn.Conv2d(128,256,kernel_size = 3, stride = 1 , padding = 1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.MaxPool2d(2,2),
            nn.Conv2d(256,512,kernel_size = 3, stride = 1 , padding = 2),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.MaxPool2d(2,2),
            nn.Conv2d(512,1024,kernel_size = 3, stride = 1 , padding = 2),
            nn.BatchNorm2d(1024),
            nn.ReLU(True),
            nn.MaxPool2d(2,2))
        self.fc1 = nn.Linear(1024*2*2,512)
        self.fc2 = nn.Linear( 512,10)
        self.dropout = nn.Dropout(0.8)
        
    def forward(self, x):
        # flatten image input
        x = self.features(x)
        x = self.dropout(x)
        x = x.view(-1,1024*2*2)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = self.fc2(x)
        # add dropout layer
        return x

# initialize the NN
model = Net()
print(model)



In [ ]:
import torch.optim as optim
from torch.optim import lr_scheduler
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
        model.cuda()
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001,betas= (0.9,0.999))
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)


In [ ]:
# number of epochs to train the model
n_epochs =60

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    ###################
    # train the model #
    ###################
    model.train()
    exp_lr_scheduler.step()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss < valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), './model.pt')
        valid_loss_min = valid_loss

In [ ]:
model.load_state_dict(torch.load('model.pt'))

In [ ]:
from torch.autograd import Variable
model.eval()
test_pred = torch.LongTensor()

for i, data in enumerate(test_loader):
    data = Variable(data, volatile=True)
    if torch.cuda.is_available():
        data = data.cuda()

    output = model(data)

    pred = output.cpu().data.max(1, keepdim=True)[1]
    test_pred = torch.cat((test_pred, pred), dim=0)



In [ ]:
sub = pd.DataFrame(np.c_[np.arange(1, len(test_df)+1)[:,None], test_pred.numpy()], 
                      columns=['ImageId', 'Label'])

In [ ]:
sub.to_csv('submission.csv', index=False)